# Spojení DYNATREE dat

Skript pro dopočítání hodnot pro spojení a vyhodnocení dat. Výstupem je csv soubor. Z něj si vyfiltruj co potřebuješ a udělej jaké analýzy chceš. Pokud něco chybí nebo je potřeba opravit, vysvětlit -> R., B., ...

* vstupy: viz sekce "Načtení dat"
* data, která se přebírají předpočítaná: viz sekce "Dendrologické parametry". V tomo místě se některé sloupce přejmenovávaji.

## Výstup

* Identifikace experimentu je ve sloupcích `type`, `day`, `tree`, `measurement`. Typ je normal-mokro-den...
* Identifikace senzoru `probe`
* Sloupec `pullNo` je pořadové číslo zatažení. Číslováno od nuly, v dynamice vždy nula.  
* Sloupec `probeCam` je identifikace senzoru pro statiku. Má smysl pouze pro statiku, ale je k dispozici i pro dynamiku. Identifikuje, který inklinometr je na kameře a který ne. Sloupec nabývá hodnot  `Elasto(90)`, `InclinoCam` a `InclinoNoCam`.
* Základní frekvence `first_frequency` se přebírá ze souboru `FFT_csv_tukey.csv` (V tomto souboru je uvedena jako `peak`.)
* Hodnota `CG`, `size_category`, `DBH`, `height`, `trunk_height`, `tapering` se přebírají z xlsx souboru  `Popis_Babice_VSE.xlsx`. Tento soubor je symbolický odkaz a odkazuje na aktuální podobu xlsx souboru, která má ve jménu i datum. Hodnoty `height` a `CG` se opravují pro stromy, které prošly redukcí. Viz sekce "Dopočítání dalších sloupců".
* Sloupce `CL`, `CR`, `CT`, `dens_all`, `w_all`, `dens_e`, `w_e` se přebírají beze změny z dodaných dat (Janek). Soubor `velocity_babice_oprava.csv` je v adresáři se vstupními daty a jenom se kopíruje mezi výstupní data. Hodnoty `CL`, `CR`, `CT` jsou pro každé datum. `CR`, `CT` pro 2022-04-05 chybí. Hodnoty `dens` a `w` jsou změřeny jenom jednou za celou dobu měření. 
* Sloupce `maxima_LDD`, `hilbert_LDD`, `wavelet_LDD`, `def_LDD`, `def2_LDD`, `def2diff_LDD`, `defmulti_LDD` se přebírají z výpočtu útlumu.
  * **TODO** Vybrat jenom jednu metodu. Pokud to bude Patrikova FFT metoda (Downhill???), zařadit sem tuto metodu. Použít skripty z webové aplikace a modifikovat je.
* Sloupce `reduction_20`, `reduction_20+20`, `reduction_40` jsou True/False a určují, zda je strom po první, po druhé nebo po dvojnásobné redukci. Rozhodovací funkce je v knihovně `dynatree` a rozhoduje se podle data, typu měření a čísla stromu.
  * **TODO** Dát sem odkaz na soubor anebo přímo na funkci.
* Penetrační odpor `pressure`. Vypočteno naším skriptem z dat od Alenky a uloženo do `penetrologger.csv`.
  * **TODO** Ujasnit si, jak vyhodnotit.
* Vlhkost `w`. Vypočteno naším skriptem z dat od Alenky. 
  * **TODO** Ujasnit si, jak vyhodnotit.
* Ve sloupci `Slope` je směrnice mezi veličinami udávající náklon nebo deformaci a působící moment. O jaké veličiny se jedná se pozná podle hodnoty ve sloupci `probe`. Data ze sloupce `Slope` jsou pro pohodlnější další zpracování zopakována v jednom ze sloupců `slope_root_stiff_nocam`, `slope_root_stiff_cam` anebo `slope_stem_stiff`. Data pocházi ze souboru `anotated_regressions_static.csv`. Odstraňují se data, pro která jsou regresní koeficienty mimo zvolený limit. Limit je stanoven podle analýzy dat pomocí DBSCAN a IQR. Viz `R2_limit_M` a `R2_limit_M_Elasto`.
* Hodnoty `EdynL`, `Model1`, `Model1_CG`, `Model2`, `Model2_CG`, `I`, `Model3`,
       `Model4`, `Model5`, `Model6`, `slenderness_CG`, `slenderness_H`,
       `m_CG^2` se dopočítávají na konci tohoto skriptu.

## Obecné poznámky

* damping je jenom z extenzometrů, TODO: zdůvodnit, proč jenom extenzometry
* chybí optika statika rok 2023 a komplet optika 2024 a 2025
* 

# Načtení dat

In [1]:
import pandas as pd
import numpy as np
try:
    # Pokud je konfigurační soubor, použij jej. Pro kompilaci pomocí snakemake nebo v příkazové řádce.
    import config
    source_path = config.merge['input_dir']
    output_folder = config.merge['output_dir']
    failed_fft = config.file['FFT_failed']
except:
    # Pokud není konfigurační soubor, použij následující nastavení. Pro kompilaci v Jupyteru.
    source_path = "https://euler.mendelu.cz/dynatree/static/public"
    failed_fft = "https://raw.githubusercontent.com/arborist-mendelu/dynatree/refs/heads/master/scripts/csv/FFT_failed.csv"
    output_folder = "."
    

In [ ]:
df_description = pd.read_excel(f"{source_path}/Popis_Babice_VSE.xlsx",
                              sheet_name="Prehledova tabulka_zakludaje", index_col=1, nrows=14
                              ).drop(0)
df_description;

In [ ]:
df_dynamics = pd.read_csv(f"{source_path}/FFT_csv_tukey.csv").rename(columns={'peak':'first_frequency'})
df_dynamics = df_dynamics[df_dynamics.probe.isin(["blueMaj","yellowMaj","Elasto(90)","Pt3","Pt4","a01_z","a02_z","a03_z"])]
df_dynamics = df_dynamics[df_dynamics.tree != "JD18"]
df_dynamics;

In [ ]:
df_static = pd.read_csv(f"{source_path}/anotated_regressions_static.csv", index_col=0)
df_static = df_static[~df_static.Dependent.isin(["Force(100)","M_Pt"])]
df_static = df_static[df_static.optics==False]
df_static.Slope = np.abs(df_static.Slope)
df_static = df_static[df_static.failed == False]
df_static;

In [ ]:
# TODO: zvážit, jestli přidat Patrikovy data
df_damping = pd.read_csv(f"{source_path}/damping_factor.csv")
df_damping = df_damping[["type", "day", "tree", "probe", "measurement"]+
               [i for i in df_damping.columns if "_LDD" in i]]
df_damping = df_damping[df_damping.tree != "JD18"]
df_damping;

In [ ]:
df_vlhkosti = pd.read_csv(f"{source_path}/vlhkosti_babice.csv")
df_sondy = pd.read_csv(f"{source_path}/sondy_a_stromy.csv")
df_sondy;

In [ ]:
df_penetrologger = pd.read_csv(f"{source_path}/penetrologger.csv")
df_penetrologger;

In [ ]:
df_velocity = pd.read_csv(f"{source_path}/velocity_babice_oprava.csv")
df_velocity = df_velocity[df_velocity["tree"] != "JD18"]

# ze dne odebrat hodiny
df_velocity["day"] = df_velocity["day"].apply(lambda x:x.split( )[0])
df_velocity = df_velocity.drop(["day real", "day"], axis=1) # odebrat nepotrebne sloupce
df_velocity = df_velocity.groupby("tree").mean().reset_index() # prumer za kazdy strom
df_velocity;

# Dendrologické parametry

Přebírají se sloupce vyjmenované v proměnné `rename_columns` a sloupec `tapering`. Proměnné se přejmenovávají.

In [ ]:
rename_columns = {
    'tree no:':'tree', 
    'mass Krejza fresh (kg):':"fresh_mass",
    "kategorie":"size_category",
    "diameter at 1.3 m (cm):":"DBH",
    "height (m):": "height",
    "centre of gravity (m)*:": "CG",
    'trunk height (m):': "trunk_height"
}
df_final_description = df_description.rename(columns=rename_columns)[[i for i in rename_columns.values()] + ["tapering"]]
df_final_description["tree"] = df_final_description["tree"].map(lambda x: f"BK{x:02}")

df_final_description;

* Fresh mass je dopočítána z alometrické rovnice ze souboru `Biomass_Beech_Babice_JK`. Hodnota `mean(SB+BB)*1.73`, Hodnota 1.73 je poměr mezi hustotou dřeva v čerstvém stavu a v konvenčním stavu dle článku Zlámal a kol. 2024, Beech ....
* TODO Doplnit hmotu ze skenu.
* `height` a `CG` měřeno výškoměrem a dopočítáno. Po redukci sníženo o odpovídající procenta, viz kód níže.
* TODO Doplnit výšky ze skenů.
* `CG` je dopočítáno jako výška kmene plus polovina výšky koruny.

# Zpracování penetrologgeru a vlhkosti půdy

Pro kazdy strom nebo a pro kazdy den najdeme jednu hodnotu reprezentujici mechanicke vlastnosti pudy. 
Najdeme median pres vsechny udaje pro danou hloubku, den a strom a pote pro dany den a strom median pro vsechnhy hloubky. 
Prvni krok zajisti, ze vsechny hloubky se budou brat stejnou vahou. Jinak by byla povrchova informace vice zastoupena. 

Pracujeme s hloubkou jenom do 15 cm. Tato hloubka byla urcena po vyhodnoceni dat.

Pocita se podmineny median, pokud jsou k dispozici aspon tri hodnoty.

In [ ]:
last_column = 15

df = df_penetrologger.copy()
days = df["day"].drop_duplicates()
trees = df["tree"].drop_duplicates()
trees = [i for i in trees if (i!="JD18") & (i!="BK25")]
df = df[df["tree"].isin(trees)]

df = df[~((df["tree"] == "BK10") & (df["day"] == "2021-03-22"))]
df = df[~((df["tree"] == "BK08") & (df["day"] == "2021-03-22"))]
df = df.drop(["0","směr","poznámka","PENETRATION DATA"], axis=1)
days = df["day"].drop_duplicates().to_numpy()
df = df.loc[:,:f"{last_column}"]


# Funkce pro výpočet mediánu při alespoň 3 nenan hodnotách
def conditional_median(row):
    valid_values = row.dropna()  # Odstranění NaN hodnot
    if len(valid_values) >= 3:
        return valid_values.median()
    return np.nan  # Pokud méně než 3 hodnoty, vrátí NaN


df_means = df.groupby(["day","tree"]).agg(conditional_median)
df_final_penetro = df_means.median(axis=1, skipna=True)
df_final_penetro.name="pressure"
df_final_penetro = df_final_penetro.reset_index()
df_final_penetro;

# Přidání vlhkostí k penetrologgeru

Přidáváme první dva horizonty`

In [ ]:
df_m = df_vlhkosti.copy()
df_m = df_m [["vzorek","hmotnostní vlhkost  w", "den"]]
df_m.columns = ["vzorek","w", "den"]
# ve soupci vzorek rozdelit zapisy typu 1A na dva sloupce, sonda a horizont.
df_m["horizont"] = df_m["vzorek"].apply(lambda x: x[-1])
df_m["sonda"] = df_m["vzorek"].apply(lambda x: x[:-1]).astype(int)

# načtení tabulky pro převod mezi stromy a sondami
sondy = df_sondy
sondy;

In [ ]:
sondy["strom"] = sondy["strom"].apply(lambda x: "JD18" if x==18 else f"BK{x:02}")
df_m = df_m.merge(sondy, on='sonda', how='left')

# df_m = df_m[df_m["strom"]!="JD18"]
# df_m[df_m["den"]=="2021-03-22"]

df_m = (df_m[df_m["horizont"]
        .isin(["A","B"])]  # vybere prvni dva horizonty
        .drop(["vzorek", "horizont","sonda"], axis=1) # vynechat sloupce
        .groupby(["den","strom"]).mean()   # prumer 
        .reset_index()
       )
df_m.columns = ["day","tree","w"]
# df_m[df_m["day"]=="2021-03-22"]

In [ ]:
df_final_penetro_moisture = df_final_penetro.merge(df_m, on=["day","tree"], how='outer')
df_final_penetro_moisture = df_final_penetro_moisture[df_final_penetro_moisture.tree != "JD18"]
df_final_penetro_moisture;

Data rozdelit na tri skupiny, protoze se opakuje datum:

a) 2024-09-02, b) 2024-29-20_mokro a c) zbytek

skupina c) se bude slucovat podle datumu, skupiny a) a b) podle datumu a typu mereni.


In [ ]:
a = df_final_penetro_moisture[df_final_penetro_moisture.day == "2024-09-02"]
b = df_final_penetro_moisture[df_final_penetro_moisture.day == "2024-09-02_mokro"]
c = df_final_penetro_moisture[~df_final_penetro_moisture.day.isin(["2024-09-02","2024-09-02_mokro"])]
b.loc[:,["day","type"]] = ["2024-09-02", "mokro"]
df_final_penetro_moisture_a = a.merge(pd.DataFrame({'day':["2024-09-02"]*2, 'type':["normal","afterro2"]}), how='left')
df_final_penetro_moisture_b = b
df_final_penetro_moisture_c = c

# Tahovky převést na camera/nocamera

In [ ]:
# Limity jsou stanoveny podle analýzy dat pomocí DBSCAN a IQR
R2_limit_M = 0.95
R2_limit_M_Elasto = 0.97

df_tahovky = df_static.copy()  # zkopirovat si, aby zustala i puvodni data
df_tahovky = df_tahovky[df_tahovky["Dependent"] != "M_Pt"]  # vynechat optiku TODO: optiku nevynechavat
df_tahovky = df_tahovky[df_tahovky["Dependent"] != "Force(100)"]  # vynechat siloměr
df_tahovky = df_tahovky[df_tahovky["tree"] != "JD18"]  # vynechat jedličku
df_tahovky = df_tahovky[~df_tahovky["Slope"].isna()]  # nesmí být nan v Slope
df_tahovky = df_tahovky[df_tahovky["optics"] == False] # vynechat optiku, pro jistotu ještě jednou

# odfiltrovat spatne R^2 podle vysledku DBSCAN a hlavne IQR
mask = (df_tahovky["Dependent"] == "M") & (df_tahovky["R^2"] < R2_limit_M)
df_tahovky = df_tahovky[~mask]
mask = (df_tahovky["Dependent"] == "M_Elasto") & (df_tahovky["R^2"] < R2_limit_M_Elasto)
df_tahovky = df_tahovky[~mask]

df_tahovky["probe"] = df_tahovky["Independent"].apply(lambda x: "Elasto(90)" if x=="Elasto-strain" else x)

# Nastaveni Camera/NoCamera. Nechat i puvodni kvuli slucovani dat
mask = df_tahovky["probe"].isin(["blueMaj", "yellowMaj"])   # radky s inklinometry
df_tahovky.loc[:,"probeCam"] = df_tahovky["probe"]
df_tahovky.loc[mask,"probeCam"] = df_tahovky.loc[mask,"kamera"].apply(lambda x: "InclinoCamera" if x is True else "InclinoNoCamera" if x is False else x)  # prejmenovani

df_tahovky = df_tahovky[["type","day","tree","probe", "probeCam","measurement","pullNo","Slope"]]

# Slope kazdeho inklinometru do samostatneho sloupce
probes = ["InclinoNoCamera", "InclinoCamera", "Elasto(90)"]
columns = ["slope_root_stiff_nocam", "slope_root_stiff_cam", "slope_stem_stiff"]
for probe, column in zip(probes, columns):
    mask = df_tahovky["probeCam"]==probe
    df_tahovky.loc[mask, column] = df_tahovky.loc[mask,"Slope"]
df_tahovky;

* Slope jsou všechny směrnice a je to rozpadnuté do dalších sloupců slope_root_stiff_nocam, slope_root_stiff_cam, slope_stem_stiff

* Přidat slope pro optiku
* Přidat sloupeček, něco jako stiffness

# Finální spojení

TODO

* Co znamená v mých poznámkách (Robert) Sapflow se tremi vykricniky?
* Dodat data od Patrika pro damping pomocí FFT
* leaves True/False

In [ ]:
# df_final_penetro_moisture_a
# df_final_penetro_moisture_b
# df_final_penetro_moisture_c
# df_final_description
# df_velocity
# df_damping
# df_tahovky
# df_dynamics

In [ ]:
(df_tahovky.shape, 
 df_dynamics.shape, 
 [i for i in df_tahovky.columns if i in df_dynamics.columns],
 [i for i in df_dynamics.columns if i in df_tahovky.columns]
)

In [ ]:
# spojit tahovky a dynamiku pres spolecne sloupce
# 'type', 'day', 'tree', 'measurement', 'probe'
# Rozmery uplne nesedi, protoze tahovky nekdy byly vyhozeny a dynamika ne, nebo naopak.
DF = df_tahovky.merge(df_dynamics, how='outer')
DF.shape

In [ ]:
[i for i in DF if i in df_final_description]

In [ ]:
# Pridat popis stromu, rozmery, dendroparametry
DF = DF.merge(df_final_description, how='left')
DF;

In [ ]:
# Pridat rychlosti a data od Janka. Data pro strom. 
DF = DF.merge(df_velocity, how='left')

In [ ]:
DF;

In [ ]:
[i for i in DF.columns if i in df_damping.columns], [i for i in df_damping.columns if i in DF.columns];

In [ ]:
# pridat tlumeni podle poli ['type', 'day', 'tree', 'probe', 'measurement']
DF = DF.merge(df_damping, how='left')

In [ ]:
# Skupina c) se bude slucovat podle datumu, skupiny a) a b) podle datumu a typu mereni.
# Rozdělit na tri disjunktní skupiny, každou spojit se správnými daty a potom skupiny sesypat dohromady
# Je to kvůli tomu, že se nedá použít merge když sloupec už existuje a update mi moc nefungovalo (Robert)
mask_a = (DF.day == "2024-09-02") & (DF.type != "mokro")
mask_b = (DF.day == "2024-09-02") & (DF.type == "mokro")
mask_c = (DF.day != "2024-09-02")
DF_a = DF[mask_a]
DF_b = DF[mask_b]
DF_c = DF[mask_c]
DF_full = pd.concat(
    [DF_a.merge(df_final_penetro_moisture_a, how='left'),
     DF_b.merge(df_final_penetro_moisture_b, how='left'),
     DF_c.merge(df_final_penetro_moisture_c, how='left')
    ])
DF_full;
    

In [ ]:
# DF_full.loc[DF.day == "2024-09-02",["day","type","pressure","w"]]

# Dopočítání dalších sloupců

* Označení stupně redukce, možnosti jsou True/False a stupně redukce 20 (po první redukci), 20+20 (po druhé redukci), 40 (obě redukce současně). Rozhodovací funkce je funkce `is_reduced` z knihovny `dynatree`.
* Pro každý stupeň redukce se určí samostatně výška a poloha těžiště. Použije se vzorec s kvalifikovaným odhadem.
* Vypočítají se parametry pro dynamické korelace.

In [ ]:
from dynatree.dynatree import is_reduced

In [ ]:
for i in ["20", "20+20", "40"]:
    DF_full[f"reduction_{i}"] = DF_full.apply(lambda row: is_reduced(tree=row["tree"], day=row["day"], method=row["type"], reduction=i), axis=1)

DF_full["CG"] = DF_full["CG"].astype(float)
DF_full["height"] = DF_full["height"].astype(float)

mask = DF_full["reduction_20"] == True
DF_full.loc[mask,"height"] = DF_full.loc[mask,"height"] * 0.8
DF_full.loc[mask,"CG"] = DF_full.loc[mask,"trunk_height"] + 0.5 * (DF_full.loc[mask,"height"] - DF_full.loc[mask,"trunk_height"])

mask = DF_full["reduction_20+20"] == True
DF_full.loc[mask,"height"] = DF_full.loc[mask,"height"] * 0.8 * 0.8
DF_full.loc[mask,"CG"] = DF_full.loc[mask,"trunk_height"] + 0.5 * (DF_full.loc[mask,"height"] - DF_full.loc[mask,"trunk_height"])

mask = DF_full["reduction_40"] == True
DF_full.loc[mask,"height"] = DF_full.loc[mask,"height"] * 0.6
DF_full.loc[mask,"CG"] = DF_full.loc[mask,"trunk_height"] + 0.5 * (DF_full.loc[mask,"height"] - DF_full.loc[mask,"trunk_height"])



In [ ]:
lambda_i = 1.8751
# DF_full["m"] = DF_full["fresh_mass"]
DF_full["EdynL"] = DF_full["CL"]**2 * DF_full["dens_e"]
DF_full["Model1"] = np.sqrt(1/DF_full["height"])
DF_full["Model1_CG"] = np.sqrt(1/DF_full["CG"])

DF_full["Model2"] = DF_full.DBH/DF_full.height**2
DF_full["Model2_CG"] = DF_full.DBH/DF_full.CG**2

DF_full["I"] = np.pi * DF_full["DBH"]**4 / 64
DF_full["Model3"] = np.sqrt(DF_full["EdynL"]*DF_full["I"]/DF_full["fresh_mass"]) 
DF_full["Model4"] = DF_full["Model3"] * lambda_i**2 / (2*np.pi*(DF_full["CG"])**2)
DF_full["Model5"] = np.sqrt(DF_full["Model2"])
DF_full["Model6"] = np.sqrt(DF_full.DBH/DF_full.CG**2/DF_full["fresh_mass"])

DF_full["slenderness_CG"] = DF_full["DBH"]/DF_full["CG"]
DF_full["slenderness_H"] = DF_full["DBH"]/DF_full["height"]
DF_full["m_CG^2"] = DF_full["fresh_mass"] * DF_full["CG"]**2



# Uložit

In [ ]:
# Failed FFT
df_failed = pd.read_csv(failed_fft)

# Sloupce pro porovnání
keys = ['type', 'day', 'tree', 'measurement', 'probe']

# Nastavení NaN pro odpovídající řádky
DF_full.loc[DF_full.set_index(keys).index.isin(df_failed.set_index(keys).index), 'first_frequency'] = np.nan


In [ ]:
# Failed damping pro Elasto(90)
import requests
keys = ['type', 'day', 'tree', 'measurement', 'probe']
response = requests.get("https://euler.mendelu.cz/gallery/api/all_comments/utlum")
data = response.json()
df_failed = pd.DataFrame(data)
df_failed = df_failed["comments"].apply(pd.Series).drop(["id", "directory", "text"], axis=1)
df_failed[["day", "type", "tree", "measurement"]] = df_failed["image"].str.split('_', expand=True)
df_failed["measurement"] = df_failed["measurement"].str.replace(".png", "", regex=False)
df_failed = df_failed.drop(["image"], axis=1)
df_failed = df_failed.groupby(['day', 'type', 'tree', 'measurement']).max()
df_failed = df_failed[df_failed["rating"] <= 2]
df_failed = df_failed.reset_index()
df_failed["probe"]='Elasto(90)'
ldd_columns = [i for i in DF_full.columns if 'LDD' in i]
DF_full.loc[DF_full.set_index(keys).index.isin(df_failed.set_index(keys).index), ldd_columns] = np.nan

In [ ]:
DF_full.to_csv(f"{output_folder}/dynatree_data_merge.csv")